# 05-3. 트리의 앙상블

- 정형데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘인 앙상블 학습 알고리즘을 알아봄

In [1]:
#1
#RandomForestClassifier 클래스를 화이트 와인 분류하는 문제에 적용
#와인 데이터셋 불러오고 훈련세트,테스트세트 나눔
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar','pH']]
target = wine['class']
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
#2
#교차검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#점수 보면 훈련세트에 과대적합

0.9973541965122431 0.8905151032797809


In [ ]:
#3
#랜덤포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도 확인
rf.fit(train_input,train_target)
print(rf.feature_importances_)
#5-1의 #18번과 비교하면 당도의 중요도가 감소하고 알코올 도수와 pH 중요도가 조금 상승
#랜덤 포레스트가 특성의 일부를 랜덤하게 선택해서 결정 트리를 훈련하기 때문
#이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 됨

[0.23167441 0.50039841 0.26792718]


In [7]:
#4
#자체적으로 모델을 평가하는 점수를 얻을 수 있음
#부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB 샘플, 이는 검증세트의 역할을 함
#RandomForestClassifier 클래스의 oob_score 매개변수를 true로 지정
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


In [ ]:
#5
#엑스트라 트리 : 랜덤 포레스트와 비슷하게 동작. 100개의 결정트리를 훈련함
#랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원
#랜덤 포레스트와 다른점은 부트스트랩 샘플을 사용하지 않고 결정트리를 만들 때 전체 훈련세트 사용
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#랜덤 포레스트와 비슷한 결과 얻음

0.9974503966084433 0.8887848893166506


In [ ]:
#6
#특성 중요도, 순서는 알코올도수>당도>pH, 엑스트라 트리도 결정 트리보다 당도에 대한 의존성이 작음
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
#7
#그레이디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#거의 과대적합이 되지 않음! 결정 트리의 개수를 늘려도 과대적합에 맹 강함
#학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

0.8881086892152563 0.8720430147331015


In [17]:
#8hgb
#히스토그램 기반 그레이디언트 부스팅
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
#9
#8에서 훈련한 걸로 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input,train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
#permutation_importance = 반복하여 얻은 특성중요도 , 평균, 표준편차를 담고있음.
#평균이 랜덤 포레스트와 비슷한 비율

[0.08876275 0.23438522 0.08027708]


In [ ]:
#10
#테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)
#그래이디언트 부스팅과 비슷하게 좀 더 당도에 집중


[0.05969231 0.20238462 0.049     ]


In [20]:
#11
#HistGredientBoostingClassifier를 사용해 테스트 세트의 성능 최종적으로 확인
hgb.score(test_input, test_target)

0.8723076923076923

In [24]:
! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 11.8 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 10.4 MB/s  0:00:28m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]m1/2 [xgboost]


In [25]:
#12
#그래이디언트 부스팅 알고리즘을 구현한 또 다른 라이브러리
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb,train_input,train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [27]:
! pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.9 MB/s  0:00:00m eta 0:00:01


In [32]:
#13
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#이거 왜 이렇게 오래걸리지..?

[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Start training from score 1.140744
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM]

KeyboardInterrupt: 